In [6]:
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=10, progress_bar=True)
import pandas as pd
import os

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# 사용자 정의 Configuration

In [2]:
data_dir = '/home/dmlab/Dropbox/DATA/projects/visualize'

billboard_filepath = os.path.join(data_dir, 'billboardHot100_1999-2019.csv')
attribute_filepath = os.path.join(data_dir, 'data.csv')

save_dir = os.path.join(data_dir, 'output')
if not os.path.exists: os.makedirs(save_dir)
save_filepath = os.path.join(save_dir, 'billboard_with_attributes_%d.csv')

# Merge data
* billboardHot100_1999-2019.csv에 수록된 노래에 한하여 data.csv로부터 attributes 값을 불러옴
    - Attributes: acousticness, danceability, duration_ms, energy, explicit, instrumentalness, key, liveness, loudness, mode, popularity, speechiness, tempo, valence
* attributes 값이 없는 노래는 삭제함

In [3]:
df = pd.read_csv(billboard_filepath)

df.drop(columns=['Unnamed: 0'], inplace=True)
df['Year'] = df.Week.apply(lambda x: int(x[:4]))
print('Total number of songs: %d [%d-%d]' % (len(df), df.Year.min(), df.Year.max()))

df.head()

Total number of songs: 97225 [1999-2019]


,Artists,Name,Weekly.rank,Peak.position,Weeks.on.chart,Week,Date,Genre,Writing.Credits,Lyrics,Features,Year
0,"Lil Nas,",Old Town Road,1,1.0,7.0,2019-07-06,"April 5, 2019","Country,Atlanta,Alternative Country,Hip-Hop,Tr...","Jozzy, Atticus ross, Trent reznor, Billy ray c...","Old Town Road Remix \nOh, oh-oh\nOh\nYeah, I'm...",Billy Ray Cyrus,2019
1,"Shawn Mendes, Camila Cabello",Senorita,2,NaN,NaN,2019-07-06,"June 21, 2019",Pop,"Cashmere cat, Jack patterson, Charli xcx, Benn...",Senorita \nI love it when you call me senorita...,NaN,2019
2,Billie Eilish,Bad Guy,3,2.0,13.0,2019-07-06,"March 29, 2019","Hip-Hop,Dark Pop,House,Trap,Memes,Alternative ...","Billie eilish, Finneas","bad guy \nWhite shirt now red, my bloody nose\...",NaN,2019
3,Khalid,Talk,4,3.0,20.0,2019-07-06,"February 7, 2019","Synth-Pop,Pop","Howard lawrence, Guy lawrence, Khalid",Talk \nCan we just talk? Can we just talk?\nTa...,NaN,2019
4,"Ed Sheeran, Justin Bieber",I Don't Care,5,2.0,7.0,2019-07-06,"May 10, 2019","Canada,UK,Dance,Dance-Pop,Pop","Ed sheeran, Justin bieber, Shellback, Max mart...",I Don't Care \nI'm at a party I don't wanna be...,NaN,2019


In [7]:
df_attr = pd.read_csv(attribute_filepath, converters={'artists': eval})
attribute_columns = [item for item in df_attr.columns if item not in ['artists', 'id', 'name', 'release_date', 'year']]
df_attr['artists'] = df_attr['artists'].apply(lambda x: x[0].lower().strip())
df_attr['name'] = df_attr['name'].apply(lambda x: x.lower().strip())
def get_attributes(attribute_column, song_name, artist_name):
    song_name = song_name.lower().strip()
    artist_name = artist_name.lower().strip()
    df_attr_sub = df_attr[df_attr['name']==song_name]
    df_selected_rows = df_attr_sub[df_attr_sub['artists']==artist_name]
    if len(df_selected_rows) == 0: return None
    return df_selected_rows.iloc[0][attribute_column]

for attribute_column in attribute_columns:
    df[attribute_column] = df.parallel_apply(lambda x: get_attributes(attribute_column, x['Name'], x['Artists'].split(',')[0]), axis=1)

attribute 값이 없는 row는 삭제

In [8]:
original_number_of_rows = len(df)
df = df.dropna(subset=attribute_columns)
print('Deleted %d rows' % (original_number_of_rows - len(df)))

Deleted 29484 rows


In [9]:
filepath = save_filepath % len(df)
df.to_csv(filepath, index=False)
print('Created ', filepath)

Created  /home/dmlab/Dropbox/DATA/projects/visualize/output/billboard_with_attributes_67741.csv
